### LangGraph Assistant Configuration Demo via the SDK
This code demonstrates how to:
1. Create a new configuration of an assistant
2. Use (invoke) that assistant with a silly question
3. Update the assistant's configuration

#### Setup

In [2]:
from langgraph_sdk import get_client
from dotenv import load_dotenv
import os

load_dotenv()

# ---- SETUP ----
# Replace with your deployed LangGraph API URL and API key
DEPLOYMENT_URL = os.getenv("DEPLOYMENT_URL")  # e.g. "http://localhost:2024" or your cloud URL
API_KEY = os.getenv("API_KEY")  # If using deployed graph
GRAPH_ID = "agent"  # Name of the graph, normally set in your langgraph.json file.



#### Connect to our Deployed Agent and Create a New Assistants


In [ ]:
# 1. Connect to the LangGraph server
client = get_client(url=DEPLOYMENT_URL, api_key=API_KEY)

# 2. Create a new assistant configuration
print("Creating a new assistant configuration...")

assistant = await client.assistants.create(
    graph_id = GRAPH_ID,
    config={
        "configurable": {
            "system_prompt": "You are a helpful AI assistant that can help with research.",
            "model": "anthropic/claude-3-7-sonnet-latest",
            "selected_tools": ["get_todays_date", "advanced_research"]
        }
    },
    name="Demo Assistant"
)
print("Assistant created:", assistant)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

# 3. Create a new thread for the conversation
thread = await client.threads.create()
print("Thread created:", thread)
# 4. Use the assistant: ask a silly question
print("Invoking the assistant with a question...")
input_data = {"messages": [{"role": "human", "content": "research the latest news in the art world"}]}
# Stream the response for demonstration
async for event in client.runs.stream(
    thread["thread_id"],
    assistant["assistant_id"],
    input=input_data,
    stream_mode="updates",
):
    # Check if the event contains messages from the assistant
    if "call_model" in event.data and "messages" in event.data["call_model"]:
        for msg in event.data["call_model"]["messages"]:
            if msg["type"] == "ai":
                # Pretty print the AI message
                AIMessage(content=msg["content"], name=msg.get("name", "Assistant")).pretty_print()
            elif msg["type"] == "tool":
                ToolMessage(content=msg["content"], name=msg.get("name", "Tool")).pretty_print()
            elif msg["type"] == "human":
                # Pretty print the human message (if you want)
                HumanMessage(content=msg["content"], name=msg.get("name", "User")).pretty_print()
    else:
        # Optionally print other event types for debugging
        print(f"Event: {event.event}")
        print(event.data)
        print("\n\n")

In [ ]:
# 5. Update the assistant's configuration (e.g., change the system prompt)
print("Creating a new version for your assistant..")
updated_assistant = await client.assistants.update(
    assistant["assistant_id"],
    config={
        "configurable": {
            "model_name": "openai/gpt-4.1",
            "system_prompt": "You are a funny assistant who likes to include puns in your responses."
        }
    },
)
print("Assistant updated:", updated_assistant)

In [ ]:
# 6. Use the updated assistant

thread2 = await client.threads.create()
print("New thread created:", thread)

print("Invoking the updated assistant...")
input_data2 = {"messages": [{"role": "user", "content": "If you could be any animal, which one would you be and why?"}]}
async for event in client.runs.stream(
    thread2["thread_id"],
    updated_assistant["assistant_id"],
    input=input_data2,
    stream_mode="updates",
):
    if "call_model" in event.data and "messages" in event.data["call_model"]:
        for msg in event.data["call_model"]["messages"]:
            if msg["type"] == "ai":
                # Pretty print the AI message
                AIMessage(content=msg["content"], name=msg.get("name", "Assistant")).pretty_print()
            elif msg["type"] == "tool":
                ToolMessage(content=msg["content"], name=msg.get("name", "Tool")).pretty_print()
            elif msg["type"] == "human":
                # Pretty print the human message (if you want)
                HumanMessage(content=msg["content"], name=msg.get("name", "User")).pretty_print()
    else:
        # Optionally print other event types for debugging
        print(f"Event: {event.event}")
        print(event.data)
        print("\n\n")


In [ ]:
#7 We can also revert to a previous version if we want to
await client.assistants.set_latest(assistant['assistant_id'], 1)